<a href="https://colab.research.google.com/github/Aljawharah004/Bank-Analysis/blob/main/datadrip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## general info about cards_data.csv


In [2]:
import pandas as pd

df = pd.read_csv("cards_data.csv")

# General structure
df.info()

# Quick statistics
df.describe(include="all")

# Missing values count
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     6146 non-null   int64 
 1   client_id              6146 non-null   int64 
 2   card_brand             6146 non-null   object
 3   card_type              6146 non-null   object
 4   card_number            6146 non-null   int64 
 5   expires                6146 non-null   object
 6   cvv                    6146 non-null   int64 
 7   has_chip               6146 non-null   object
 8   num_cards_issued       6146 non-null   int64 
 9   credit_limit           6146 non-null   object
 10  acct_open_date         6146 non-null   object
 11  year_pin_last_changed  6146 non-null   int64 
 12  card_on_dark_web       6146 non-null   object
dtypes: int64(6), object(7)
memory usage: 624.3+ KB


,0
id,0
client_id,0
card_brand,0
card_type,0
card_number,0
expires,0
cvv,0
has_chip,0
num_cards_issued,0
credit_limit,0


In [3]:
# Check basic info
print("Shape:", df.shape)  # (rows, columns)
print(df.head())  # First 5 rows
print(df.columns)  # Column names

Shape: (6146, 13)
     id  client_id  card_brand        card_type       card_number  expires  \
0  4524        825        Visa            Debit  4344676511950444  12/2022   
1  2731        825        Visa            Debit  4956965974959986  12/2020   
2  3701        825        Visa            Debit  4582313478255491  02/2024   
3    42        825        Visa           Credit  4879494103069057  08/2024   
4  4659        825  Mastercard  Debit (Prepaid)  5722874738736011  03/2009   

   cvv has_chip  num_cards_issued credit_limit acct_open_date  \
0  623      YES                 2       $24295        09/2002   
1  393      YES                 2       $21968        04/2014   
2  719      YES                 2       $46414        07/2003   
3  693       NO                 1       $12400        01/2003   
4   75      YES                 1          $28        09/2008   

   year_pin_last_changed card_on_dark_web  
0                   2008               No  
1                   2014          

### cards_data.csv errors check


In [4]:
import pandas as pd

df = pd.read_csv("cards_data.csv")

# Remove $ and convert to numeric (NaN if invalid)
df["credit_limit"] = pd.to_numeric(df["credit_limit"].str.replace("$", "", regex=False), errors="coerce")

# Find invalid rows
invalid_rows = df[df["credit_limit"].isna() | (df["credit_limit"] < 0)]

print(invalid_rows)

Empty DataFrame
Columns: [id, client_id, card_brand, card_type, card_number, expires, cvv, has_chip, num_cards_issued, credit_limit, acct_open_date, year_pin_last_changed, card_on_dark_web]
Index: []


## general info about users_data.csv


In [5]:
import pandas as pd

df = pd.read_csv("users_data.csv")

# General structure
df.info()

# Quick statistics
df.describe(include="all")

# Missing values count
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB


,0
id,0
current_age,0
retirement_age,0
birth_year,0
birth_month,0
gender,0
address,0
latitude,0
longitude,0
per_capita_income,0


### userss_data.csv errors check

In [6]:
# Remove $ and commas, handle parentheses as negative values
for col in ["per_capita_income", "yearly_income", "total_debt"]:
    df[col] = (
        df[col]
        .astype(str)  # ensure string
        .str.replace("$", "", regex=False)  # remove dollar sign
        .str.replace(",", "", regex=False)  # remove commas
        .str.replace(r"\((.*?)\)", r"-\1", regex=True)  # convert (123) to -123
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")  # convert to numeric

## general info about transactions_data.csv

In [7]:
import pandas as pd

df = pd.read_csv("transactions_data.csv")

# General structure
df.info()

# Quick statistics
df.describe(include="all")

# Missing values count
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155901 entries, 0 to 155900
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              155901 non-null  int64  
 1   date            155901 non-null  object 
 2   client_id       155901 non-null  int64  
 3   card_id         155901 non-null  int64  
 4   amount          155901 non-null  object 
 5   use_chip        155900 non-null  object 
 6   merchant_id     155900 non-null  float64
 7   merchant_city   155900 non-null  object 
 8   merchant_state  138733 non-null  object 
 9   zip             137951 non-null  float64
 10  mcc             155900 non-null  float64
 11  errors          2433 non-null    object 
dtypes: float64(3), int64(3), object(6)
memory usage: 14.3+ MB


,0
id,0
date,0
client_id,0
card_id,0
amount,0
use_chip,1
merchant_id,1
merchant_city,1
merchant_state,17168
zip,17950


In [8]:
# Drop rows where any of the specified columns are null
df_clean = df.dropna(subset=["client_id", "card_id", "amount", "use_chip", "mcc"])

# Check the result
print(f"Rows after cleaning: {len(df_clean)}")

Rows after cleaning: 155900


### transactions_data.csv errors check

In [9]:
# Clean amount column
df_clean["amount"] = (
    df_clean["amount"]
    .astype(str)  # ensure it's string
    .str.replace("$", "", regex=False)  # remove dollar sign
    .str.replace(",", "", regex=False)  # remove commas if any
    .str.replace(r"\((.*?)\)", r"-\1", regex=True)  # convert (123.45) to -123.45
)

# Convert to numeric
df_clean["amount"] = pd.to_numeric(df_clean["amount"], errors="coerce")

# Check
print(df_clean["amount"].head(20))

0     -77.00
1      14.57
2      80.00
3     200.00
4      46.41
5       4.81
6      77.00
7      26.46
8     261.58
9      10.74
10      3.51
11      2.58
12     39.63
13     43.33
14     49.42
15      1.09
16     73.79
17    100.00
18     26.04
19    -64.00
Name: amount, dtype: float64


/tmp/ipython-input-1361183022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["amount"] = (
/tmp/ipython-input-1361183022.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["amount"] = pd.to_numeric(df_clean["amount"], errors="coerce")


### Creating MCC mapping list

In [ ]:
import pandas as pd

# Create a MCC mapping list (add more if needed)
mcc_data = [
    {"mcc": "5813.0", "mcc_name": "Drinking Places (Alcoholic Beverages)"},
    {"mcc": "5499.0", "mcc_name": "Misc. Food Stores"},
    {"mcc": "5311.0", "mcc_name": "Department Stores"},
    {"mcc": "4829.0", "mcc_name": "Money Transfer"},
    {"mcc": "5942.0", "mcc_name": "Book Stores"},
    {"mcc": "4784.0", "mcc_name": "Tolls and Bridge Fees"},
    {"mcc": "7801.0", "mcc_name": "Government Licensing"},
]

# Save mapping as CSV
mcc_lookup = pd.DataFrame(mcc_data)
mcc_lookup.to_csv("mcc_lookup_mastercard.csv", index=False)

print("MCC mapping file created.")


MCC mapping file created.


### Mapping MCC categories

In [ ]:
import pandas as pd


# Load your MCC lookup
mcc_lookup = pd.read_csv("mcc_lookup_mastercard.csv")  # has columns mcc, mcc_name

# Make sure MCC codes are strings & padded to 4 digits
df_clean["mcc"] = df_clean["mcc"].astype(str).str.zfill(4)
mcc_lookup["mcc"] = mcc_lookup["mcc"].astype(str).str.zfill(4)

# Merge directly
tx_enriched = df_clean.merge(mcc_lookup, on="mcc", how="left")

# Fill missing MCC names
tx_enriched["mcc_name"] = tx_enriched["mcc_name"].fillna("Unknown MCC")

# 6Add your bucket logic
bucket_map = {
    "5813.0": "Food & Drink",
    "5499.0": "Groceries",
    "5311.0": "Retail",
    "4829.0": "Financial",
    "5942.0": "Retail",
    "4784.0": "Transportation",
    "7801.0": "Government"
}
tx_enriched["mcc_bucket"] = tx_enriched["mcc"].map(bucket_map).fillna("Other")

# Save result
tx_enriched.to_csv("transactions_with_mcc_names.csv", index=False)

print("Saved with MCC names and buckets.")


/tmp/ipython-input-2330964847.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["mcc"] = df_clean["mcc"].astype(str).str.zfill(4)


Saved with MCC names and buckets.


In [ ]:
tx_enriched.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,mcc_name,mcc_bucket
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935.0,Beulah,ND,58523.0,5499.0,NaN,Misc. Food Stores,Groceries
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570.0,Bettendorf,IA,52722.0,5311.0,NaN,Department Stores,Retail
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092.0,Vista,CA,92084.0,4829.0,NaN,Money Transfer,Financial
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092.0,Crown Point,IN,46307.0,4829.0,NaN,Money Transfer,Financial
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051.0,Harwood,MD,20776.0,5813.0,NaN,Drinking Places (Alcoholic Beverages),Food & Drink


-------------------------------------------------------------------------------------


### another way


In [11]:
import pandas as pd

# Load your transactions data
transactions = pd.read_csv('transactions_data.csv')  # replace with your file path

# Load MCC codes
mcc_codes = pd.read_csv('mcc_codes.csv')  # your GitHub CSV

# Keep relevant columns from MCC file (both edited and combined descriptions)
mcc_codes = mcc_codes[['mcc', 'edited_description', 'combined_description']]

# Merge transactions with MCC descriptions
transactions = transactions.merge(mcc_codes, on='mcc', how='left')

# Optional: rename columns for clarity
transactions.rename(columns={
    'edited_description': 'mcc_edited_description',
    'combined_description': 'mcc_combined_description'
}, inplace=True)

# Preview
print(transactions.head())

# Save enriched transactions
transactions.to_csv("transactions_with_mcc_names.csv", index=False)

        id                 date  client_id  card_id   amount  \
0  7475327  2010-01-01 00:01:00       1556     2972  $-77.00   
1  7475328  2010-01-01 00:02:00        561     4575   $14.57   
2  7475329  2010-01-01 00:02:00       1129      102   $80.00   
3  7475331  2010-01-01 00:05:00        430     2860  $200.00   
4  7475332  2010-01-01 00:06:00        848     3915   $46.41   

            use_chip  merchant_id merchant_city merchant_state      zip  \
0  Swipe Transaction      59935.0        Beulah             ND  58523.0   
1  Swipe Transaction      67570.0    Bettendorf             IA  52722.0   
2  Swipe Transaction      27092.0         Vista             CA  92084.0   
3  Swipe Transaction      27092.0   Crown Point             IN  46307.0   
4  Swipe Transaction      13051.0       Harwood             MD  20776.0   

      mcc errors                             mcc_edited_description  \
0  5499.0    NaN  Misc. Food Stores – Convenience Stores and Spe...   
1  5311.0    NaN      